<a href="https://colab.research.google.com/github/VIVEK818713/Feature-Engineering/blob/main/3_Column_Transformer_%2B_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ColumnTransformation

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/0machine learning/csv data/covid.csv')
df.head()

##Now understand the data and after that use column transfromation on desired column

In [ ]:
df['city'].value_counts()

In [ ]:
df['cough'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

## here we will not work on 'age' column. 
## we will apply nominal categorical encoding on 'cough' by using OrdinalEncoder
## we will apply OneHotEncoding on column 'gender' and 'city' by using OneHotEncoder
## you can do Label encoding on target column'has_covid' but we are not do it now in this topic.
## and we will handle missing value using simpleimputer, we will discuss simple imputer later on. simple imputer is using for filling the missing value.

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

##without using column transformation

In [ ]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape  

In [ ]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

In [ ]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

##we are removing 'age' column and convert it to nparray then concatenate it with all other columns

In [ ]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

In [ ]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

##Now using column Transformation

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [ ]:
transformer.fit_transform(X_train)

In [ ]:
transformer.transform(X_test)

In [ ]:
print(transformer.fit_transform(X_train).shape)
print(transformer.transform(X_test).shape)

#Pipeline

In [ ]:
import pandas as pd
import numpy as np
import os  

In [ ]:
os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/Colab Notebooks/Kaggle json'

In [ ]:
!kaggle datasets download -d hesh97/titanicdataset-traincsv

In [ ]:
! unzip /content/titanicdataset-traincsv.zip

In [ ]:
df=pd.read_csv('/content/train.csv')
df.head()

In [ ]:
df.shape

##work without pipeline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

##we are droping some column because these column is not needed for our analysis , these columns are 'PassengerId','Name','Ticket','Cabin'

In [ ]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)  

In [ ]:
df.head()

##Now spilt our data into train and test split

In [ ]:
# Step 1 -> train/test/split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

##Now Check whether there is any missing values or not in any columns

In [ ]:
df.isnull().sum()

##there are missing values in 'age' and 'Embarked' column so we are using simple imputer to fill this missing values. in age we are simpleimputer using 'mean' as default and 'embarked' column we use strategy 'most-frequent'.

In [ ]:
# Applying imputation

si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [ ]:
X_train_embarked

In [ ]:
X_train_age

##Now apply one hot encoding on column 'sex' and 'embarked'

In [ ]:
# one hot encoding Sex and Embarked

ohe_sex = OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False,handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

In [ ]:
X_train_embarked

In [ ]:
X_train.head(2)

## Now we have 3 array namely x_train_age,x_train_sex, and x-train_embarked. and we need rest of columns as it is so we will create a array in which we have rest of columns. 
## for that we will drop 3 columns sex,age,embarked because we have now 3 new array for this columns

In [ ]:
X_train_rem = X_train.drop(columns=['Sex','Age','Embarked'])

In [ ]:
X_test_rem = X_test.drop(columns=['Sex','Age','Embarked'])

##now concatenate all 4 array X_train_rem,x-train_age,x_train_sex, and x-train_embarked

In [ ]:
X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)  

In [ ]:
X_train_transformed

##Now use decision tree

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

In [ ]:
y_pred = clf.predict(X_test_transformed)
y_pred

##Now check accuracy

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

##above we did not do column transformation that's why our work is too lenghty. we pick each column and do the desired work.
##now assume that we have a html form where we can enter new passanger data and our model will predict that passanger will survived or not.

##for exporting the model we have to import pickle

In [ ]:
import pickle  

##we have to use pickle.dump(classifer name,open(location,mode)
##and we have to use ohe_sex and ohe_embarked also because at the run time when new data is given by user then he/she will enter 'male/female' for sex entry and emabrked value like's','c' or 'q' .
##so we have to do one hot encoding of this entry data that's why we are using ohe_sex and ohe_embarked .

In [ ]:
pickle.dump(ohe_sex,open('ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('ohe_embarked.pkl','wb'))
pickle.dump(clf,open('clf.pkl','wb'))  

##But how to use these model?
##lets consider we have a page where we have to give data and predict the output.
##for this purpose we will use another step of coding in this file as below.

In [ ]:
import pickle
import numpy as np

##now load the model. as you encode here we will decode .
##use pickle.load()

In [ ]:
ohe_sex = pickle.load(open('ohe_sex.pkl','rb'))
ohe_embarked = pickle.load(open('ohe_embarked.pkl','rb'))
clf = pickle.load(open('clf.pkl','rb'))

In [ ]:
# Assume user input
# Pclass/gender/age/SibSp/Parch/Fare/Embarked
test_input = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [ ]:
test_input

##here we have to do all the preprocessing steps as we done previously like imputer,onehot encoder then concatenate and then prediction. here you should not do imputer beacause data is provided by you and there is no missing value.

In [ ]:
test_input_sex = ohe_sex.transform(test_input[:,1].reshape(1,1))

In [ ]:
test_input_sex
# output 0,1 because it is a 'male'

In [ ]:
test_input_embarked = ohe_embarked.transform(test_input[:,-1].reshape(1,1))

In [ ]:
test_input_embarked

In [ ]:
test_input_age = test_input[:,2].reshape(1,1)

In [ ]:
test_input_age

##now concatenate rest columns with test_input_age,test_input_sex,test_input_embarked.

In [ ]:
test_input_transformed = np.concatenate((test_input[:,[0,3,4,5]],test_input_age,test_input_sex,test_input_embarked),axis=1)

In [ ]:
test_input_transformed

In [ ]:
test_input_transformed.shape

##Now predict

In [ ]:
clf.predict(test_input_transformed)

##above code is your production code. you can think we have do alot of work for that
##but but but if we use pipeline then this work will be reduced in the next section we will use pipeline.
##understand the thing if you are not using pipeline ...
##steps of series you are doing in the traing time that series of steps you have write in your production code. if change a little bit in traning part then you have to change in production also.
##now lets see if you used pipeline what will happen..

##titanic-using-pipeline

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn. feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [ ]:
os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/Colab Notebooks/Kaggle json'

In [ ]:
!kaggle datasets download -d hesh97/titanicdataset-traincsv

In [ ]:
! unzip /content/titanicdataset-traincsv.zip

In [ ]:
df=pd.read_csv('/content/train.csv')
df.head()

In [ ]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

#How we will make our pipeline ? What is our Plan?
##The first step of our pipeline is impute the missing value in age and embarked using one column transform , the out put of this we will use an input to next column transform which work on onehot encoding on column sex and embarked. The output of this will be input to next column transformer which is scaling(it may be it is useful or not but for understanding of pipeline we will do this step). The out of this column transformer will be the input to next step which is feature selcetion(means we will take best 5 or 8 feature for our consideration, it may be result goes worst but for understanding pipeline we including this beacuse it may be in any case you need it in future).
##After that we will train our model using Decision tree algorithm.

In [ ]:
from pandas.core.common import random_state
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],
                                               test_size=0.2,random_state=42)

##first column transformer creation for missing value imputation
##we use index no. instead of column name because output of this transformer is not a dataframe but it is an array so you know that numpy don't understand column name it understand only index no.
##we don't want to drop rest columns so we use remainder='passthrough'

In [ ]:
trf1=ColumnTransformer([
    ('Impute_Age',SimpleImputer(),[2]),
    ('Impute_Embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

##Next column transformer for one hot encoding on sex and embarked, we are not using drof first parameter because we are working on Decision tree

In [ ]:
trf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

## Our next column transformer is scaling. here we are not using standered scalere because we will do feature selection in the next step and in feature seection we use Min Max Scaler.
## we are appling MinMax scaler on all the columns if you want to skip categorical column you can do it but we are taking all columns in this case.
##so we use slice(0,10) slice will apply minmax on this range. but why 0 to 10
## understand this we have 7 columns
## we do one hot encoding on age so it will give 2 new columns
## and embarked will produce 3 new columns
##**total 5 new columns**
## but we have to remove age and embarked from the 7 columns so we have here 5 columns 
## so **5 plus 5 is 10**

In [ ]:
trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

##next step is feature selection
##we dont need to apply column transformer
##here you should understand how you can insert feature selection step in your pipeline .here we are nor work on how feature selection work.

In [ ]:
trf4=SelectKBest(score_func=chi2,k=8)

##Train the model

In [ ]:
trf5=DecisionTreeClassifier()

#Now Create the Pipeline

In [ ]:
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

##Now we have our pipeline object and now we have to fit it on X_train and y_train
##how pipeline work on X_train. first X_train goes to trf1 and output go to trf2 ,then its output go to trf3, then its output go to trf4 then its output go to trf5.
##if in your pipe line there is no model trainig then you will use pipe.fit_transform because we are not tarin our model but in our case we are tarining our model so we use pipe.fit(X_train,y_train). so that we can call predict().

In [ ]:
pipe.fit(X_train,y_train)

##if you want to display your pipeline in a good mannaer then use below code and then fit it

In [ ]:
from sklearn import set_config
set_config(display='diagram')

In [ ]:
pipe.fit(X_train,y_train)

## Now understand how it works and how you can extract information

In [ ]:
pipe.named_steps

In [ ]:
pipe.named_steps['trf1']

In [ ]:
pipe.named_steps['trf1'].transformers_

In [ ]:
pipe.named_steps['trf1'].transformers_[0]

In [ ]:
pipe.named_steps['trf1'].transformers_[0][1]

In [ ]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

In [ ]:
# if you want to check
pipe.named_steps['trf1'].transformers_[1][1].statistics_ 

In [ ]:
y_pred=pipe.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

##accuracy is down it may be because of feature selection step you can chek it by removing feature seletion step from the pipeline it may it works.
#cross validation using pipeline
##it may be you dont know about cross validation but here we are doing a complete job related to pipeline. it may be you use cross validation in future so here we are adding these below stpes also for the understanding purpose.

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

#gridsearch using pipeline
##in the future you will use a term namely hyper parameter tuning means you will tune your parameter for the best value of the parameter so we use gridsearch.
##in our case we are using Decision tree so we are using max depth parameter and tune it for best value. lets check it our model name is trf5.

In [ ]:
param={
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pipe,param,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

#exporting the pipeline
##what if we want to use this pipeline in production so step is here

In [ ]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

##now code for production

In [ ]:
import pickle
import numpy as np

In [ ]:
pipe=pickle.load(open('pipe.pkl','rb'))

In [ ]:
test_data=np.array([2,'male',31.0,0,0,10.5,'S'],dtype=object).reshape(1,7)

In [ ]:
pipe.predict(test_data)

#in future if you change any coding then you need not to change the production code it will reamin unchanged.
##you have change your code and udate your new pickle file on the server after modification in your file. your production code will be remain unchanged.